In [1]:
# Load model  directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch import nn
from transformers import TrainingArguments, Trainer
from torch.utils.data import Dataset
import torch
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import accelerate
import transformers
import torch.nn.functional as F
# from torch.utils.data import DataLoader

2024-05-24 10:49:23.939733: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-24 10:49:23.939834: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-24 10:49:24.093694: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
import random

In [3]:
MAX_SEQUENCE_LENGTH = 128
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [4]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    if len(logits) != len(labels):
        print("Issue in Compute Metrics")
    predictions = np.argmax(logits, axis=-1)  # This gives predictions for each sample
    return {'accuracy': accuracy_score(labels , predictions)}

In [5]:
def get_data_lists(data_dict, tweet_class = -1):
    """returns the list of tweets and np array of labels"""
    comments = []
    labels = []
    
    # tweet class 1 for hate  , 2 for non-hate
    

    for item in data_dict:
        
        if tweet_class == 1 and item['Label'] != 'hate': 
            continue
            
            
        if tweet_class == 2 and item['Label'] == 'hate':
            continue
            
        
        comments.append(item['Comment'])
        labels.append(1 if item['Label'] == 'hate' else 0)
        
    return comments,labels

In [6]:
# def get_data_lists_integer_labels(data_dict, label_needed):
#     """returns the list of tweets and np array of labels"""
#     comments = []
#     labels = []
    
#     # tweet class 1 for hate  , 2 for non-hate
    

#     for item in data_dict:
        
#         if label_needed != item['Label']:
#             continue
                        
        
#         comments.append(item['Comment'])
#         labels.append(item['Label'])
        
#     return comments,labels

In [7]:
def count_parameters(model):
    ans = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f" # of Trainable Parameters: {ans} ")

In [8]:
def freeze_partially(model):
    for param in model.parameters():
        param.requires_grad = False  # Initially freeze all parameters

    # Unfreeze the hidden dense layer (before final layer)
    # RoBERTa
    # X -----  [] ------ [] ------- []----[]-----> output
    for param in model.roberta.encoder.layer[-1].attention.self.query.parameters():
        param.requires_grad = True
    for param in model.roberta.encoder.layer[-1].attention.self.key.parameters():
        param.requires_grad = True
    for param in model.roberta.encoder.layer[-1].attention.self.value.parameters():
        param.requires_grad = True
    for param in model.roberta.encoder.layer[-1].attention.output.dense.parameters():
        param.requires_grad = True
    for param in model.roberta.encoder.layer[-1].attention.output.LayerNorm.parameters():
        param.requires_grad = True
    for param in model.roberta.encoder.layer[-1].intermediate.parameters():
        param.requires_grad = True
    for param in model.roberta.encoder.layer[-1].output.dense.parameters():
        param.requires_grad = True
    for param in model.roberta.encoder.layer[-1].output.LayerNorm.parameters():
        param.requires_grad = True

In [9]:
tokenizer = AutoTokenizer.from_pretrained("facebook/roberta-hate-speech-dynabench-r4-target")
model = AutoModelForSequenceClassification.from_pretrained("facebook/roberta-hate-speech-dynabench-r4-target" )
# model = model.to(device)

# text -> tokenizer -> 10294 (numerical form) -> Model


tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/816 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [ ]:
# comment = "this is a text"


# encodings = tokenizer.encode_plus(comment,
#                                         add_special_tokens = True, 
#                                         truncation=True,
#                                         padding='max_length',
#                                         max_length=MAX_SEQUENCE_LENGTH, 
#                                         return_tensors='pt') 

# ids , att = encodings['input_ids'].to(device) , encodings['attention_mask'].to(device)
# base_model(ids , att)

In [ ]:
# class CustomModel(nn.Module):
#     def __init__(self, base_model):
#         self.base_model = base_model
#         self.tokenizer = tokenizer


#     def forward(self, input_ids, attention_mask=None, labels=None):
#         return base_model.forward(input_ids , attention_mask , labels)
    
#     def inference(self, comment):
#         encodings = tokenizer.encode_plus(comment,
#                                                 add_special_tokens = True, 
#                                                 truncation=True,
#                                                 padding='max_length',
#                                                 max_length=MAX_SEQUENCE_LENGTH, 
#                                                 return_tensors='pt') 
        
#         ids , att = encodings['input_ids'].to(device) , encodings['attention_mask'].to(device)
#         return self.base_model(ids , att)
        

In [ ]:
# class CustomModel:
#     def __init__(self,base_model):
       
#         self.base_model = base_model

        

#     def inference(self, text):
#         # Tokenize the input text
#         encodings = tokenizer.encode_plus(
#             text,
#             add_special_tokens=True, 
#             truncation=True,
#             padding='max_length',
#             max_length=128, 
#             return_tensors='pt'
#         )

#         # Move the input tensors to the appropriate device
#         ids, att = encodings['input_ids'].to(self.device), encodings['attention_mask'].to(device)
        
#         # Perform inference
#         with torch.no_grad():
#             outputs = self.base_model(input_ids=ids, attention_mask=att)

#         return outputs


In [ ]:
# model = CustomModel(base_model)


In [ ]:
# model.inference('this is a comment')

In [ ]:
# class CustomModel(nn.Module):
#     def __init__(self, base_model):
#         super(CustomModel, self).__init__()
#         self.base_model = base_model
        
#         # Replace the out_proj layer in the existing classifier
#         self.base_model.classifier.out_proj = nn.Identity()
#         self.final_layer = nn.Linear(768, 3)  # Your custom final layer with 3 units

#     def forward(self, input_ids, attention_mask=None, labels=None):
#         # Get the outputs from the RoBERTa base model
#         outputs = self.base_model.roberta(
#             input_ids=input_ids,
#             attention_mask=attention_mask
#         )
        
#         # Get the [CLS] token hidden state
#         sequence_output = outputs[0]
#         cls_output = sequence_output[:, 0, :]  # Get the [CLS] token hidden state
        
#         # Pass the [CLS] token hidden state through the classifier dense layer and dropout
#         cls_output = self.base_model.classifier.dense(cls_output)
#         cls_output = self.base_model.classifier.dropout(cls_output)
        
#         # Pass through the custom final layer
#         logits = self.final_layer(cls_output)
        
#         loss = None
#         if labels is not None:
#             loss_fct = nn.CrossEntropyLoss()
#             loss = loss_fct(logits.view(-1, 3), labels.view(-1))
        
#         return (loss, logits) if loss is not None else logits



In [10]:
# model = CustomModel(base_model)
# model = base_model

print(model)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [11]:
df = pd.read_csv('/kaggle/input/shortyetgoodset/Book.csv', encoding='latin-1')
data_dict = df.to_dict('records')

# comments_list_1_hate , labels_list_1_hate = get_data_lists(data_dict, 1)
# comments_list_1_nonhate , labels_list_1_nonhate = get_data_lists(data_dict, 2)
comments_1 , labels_1 = get_data_lists(data_dict)
print(len(comments_1))

40623


In [ ]:
# df = pd.read_csv('/kaggle/input/dyn-gen-hate-speech/dyn_set.csv', encoding='latin-1')
# data_dict = df.to_dict('records')
# # comments_list_2_hate , labels_list_two_hate = get_data_lists(data_dict, 1)
# # comments_list_2_nonhate , labels_list_two_nonhate = get_data_lists(data_dict, 2)

# comments_2 , labels_2 = get_data_lists(data_dict)
# len(comments_2)

In [ ]:
# df = pd.read_csv('/kaggle/input/comments-positive-neutral-negative/file.csv', encoding='latin-1')
# data_dict = df.to_dict('records')

# comments_list_3_hate , labels_list_3_hate = get_data_lists_integer_labels(data_dict, 1)
# len(comments_list_3_hate)

In [12]:
comments_list, labels_list = comments_1, labels_1

paired_data = list(zip(comments_list, labels_list))
random.shuffle(paired_data)
shuffled_comments, shuffled_labels = zip(*paired_data)

subset_size = 20000
comments_list = shuffled_comments[:subset_size]
labels_list = shuffled_labels[:subset_size]

In [13]:
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        
        self.labels = torch.tensor(labels, dtype=torch.long)
        self.encodings = tokenizer(texts,add_special_tokens = True,  truncation=True, padding='max_length', max_length=MAX_SEQUENCE_LENGTH, return_tensors='pt') 
        # all examples -> 138438
    
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        
        return {
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask': self.encodings['attention_mask'][idx],
            'labels': self.labels[idx]
        }
    
    # return one example (numerical form) -> Model 
    
    # some_examples -> DataSet<Some_exampls> -> DataSet[4] -> example[4]
    
        

In [14]:
from random import shuffle

# Pair up the comments and labels
paired_data = list(zip(comments_list, labels_list))

# Shuffle the paired data
shuffle(paired_data)

# Unzip the shuffled paired data
shuffled_comments, shuffled_labels = zip(*paired_data)

total_examples = len(shuffled_comments) # 100

train_size = int(0.8 * total_examples) # 80 
val_size = int(0.1 * total_examples)   # 10
test_size = total_examples - train_size - val_size # 10

TrainDataSet = CustomDataset(shuffled_comments[:train_size], shuffled_labels[:train_size], tokenizer)
TestDataSet = CustomDataset(shuffled_comments[train_size:train_size+val_size], shuffled_labels[train_size:train_size+val_size], tokenizer)
ValidationDataSet = CustomDataset(shuffled_comments[train_size+val_size:], shuffled_labels[train_size+val_size:], tokenizer)



print(total_examples)
print(len(TrainDataSet) + len(TestDataSet) + len(ValidationDataSet))

20000
20000


In [ ]:
# model = model.to(device)

In [15]:
# 125 Million Paramters
freeze_partially(model)
count_parameters(model)
# 70 Lakh -> values to change 


 # of Trainable Parameters: 7087872 


In [16]:

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    save_steps=500,                    # save a checkpoint every 500 steps
    save_total_limit=2,                # only keep the last 2 checkpoints
    load_best_model_at_end=True,       # load the best model at the end of training
    evaluation_strategy='steps',
    metric_for_best_model='accuracy'  # Use accuracy for tracking the best model
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=TrainDataSet,          # training dataset
    eval_dataset=ValidationDataSet,      # validation dataset
    compute_metrics=compute_metrics,     # function to compute metrics   # Accuracy 
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [17]:
# Train the model
results = trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss,Accuracy
500,0.301400,0.237268,0.919500
1000,0.227500,0.228964,0.922000
1500,0.239300,0.233428,0.920500
2000,0.226500,0.228506,0.922500
2500,0.229600,0.224209,0.922000
3000,0.217500,0.227461,0.924000


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

In [22]:
print(results)

TrainOutput(global_step=3000, training_loss=0.24029634602864583, metrics={'train_runtime': 604.4428, 'train_samples_per_second': 79.412, 'train_steps_per_second': 4.963, 'total_flos': 3157332664320000.0, 'train_loss': 0.24029634602864583, 'epoch': 3.0})


In [23]:
model.training = False

# Evaluate the model on the test dataset
test_results = trainer.evaluate(TestDataSet)


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


In [24]:
test_results['eval_accuracy']

0.923

In [32]:
model.save_pretrained('/kaggle/working/model_files' , safe_serialization = True)

In [31]:


# inp = tokenizer.encode_plus(text,
#                             add_special_tokens = True, 
#                             truncation=True,
#                             padding='max_length',
#                             max_length=MAX_SEQUENCE_LENGTH,
#                             return_tensors='pt') 

# ids = inp['input_ids'].to(device)
# att = inp['attention_mask'].to(device)


# output = model.forward(ids, att)

# output.logits

tensor([[ 0.7959, -0.4124]], device='cuda:0', grad_fn=<AddmmBackward0>)

tensor([[-4.4369,  3.9192]], device='cuda:0', grad_fn=<AddmmBackward0>)